### Day 4:

In [1]:
# Load input
with open("inputs/day4_input.dat","r") as f:
    data = f.read().splitlines() 

In [2]:
# Loop through the input and get it in a format we can use
# 1D boards are fine
boards = []
for ix, line in enumerate(data):
    # First line is the bingo number sequence:
    if ix == 0:
        bingo_numbers = [int(x) for x in data[ix].split(',')]
    elif ix == 1:
        new_board = []
    elif line == '':
        boards.append(new_board)
        new_board = []
    else:
        # Single digits have two spaces
        nums = line.strip().replace('  ',' ').split(' ')
        nums = [int(x) for x in nums]
        new_board.extend(nums)

# And add the last board
boards.append(new_board)
        
n_boards = len(boards)
size_boards = len(boards[0])

In [3]:
print(n_boards)
print(size_boards)

100
25


### Part 1:
- Find the first winning board when the numbers are called out one at a time
- Calculate the score as the sum of unmarked numbers on that board multiplied by last number called

Plan:
- Class with array for board.
    - Only need unmarked numbers at end so can turn to -1 when a number comes up
    - Check by summing rows/columns and looking for -5
    - Probably slow but only 100 inputs so should be doable

In [4]:
class BingoBoard(object):
    def __init__(self,board,board_num=0):
        #board is a flattened 5x5 bingo board
        self.board = board
        self.size = 5
        self.board_num = board_num
    
    def mark_number(self, num):
        self.board = [-1 if x is num else x for x in self.board]
    
    def check_bingo(self, verbose=True):
        # Rows (e.g. 0,1,2,3,4)
        for ix in range(self.size):
            row = self.board[ix*self.size:(ix+1)*self.size]
            if sum(row) == -self.size:
                if verbose:
                    print("Bingo!")
                return True
        
        # Columns (e.g. 0,5,10,15,20)
        for ix in range(self.size):
            col = self.board[ix::self.size]
            if sum(col) == -self.size:
                if verbose:
                    print("Bingo!")
                return True
        return False
    
    def remaining_sum(self):
        score = 0
        for num in self.board:
            if num != -1:
                score += num
        return score

In [5]:
# Test it out on a board.
board = BingoBoard(boards[0])
print(board.board)
for x in [14,85,51,21,41]: # First column bingo
    print("Adding",x)
    board.mark_number(x)
    board.check_bingo()
# It works!

[14, 33, 79, 61, 44, 85, 60, 38, 13, 48, 51, 34, 11, 19, 7, 21, 30, 73, 6, 76, 41, 4, 65, 18, 91]
Adding 14
Adding 85
Adding 51
Adding 21
Adding 41
Bingo!


In [6]:
# Make all the boards:
all_boards = []
for board in boards:
    all_boards.append(BingoBoard(board))

In [7]:
# Loop through the input numbers and check them all
for ix,num in enumerate(bingo_numbers):
    for board_ix in range(n_boards):
        board = all_boards[board_ix]
        board.mark_number(num)
        has_bingo = board.check_bingo()
        all_boards[board_ix] = board
        if has_bingo:
            break
    
    # Need to break out of 2 loops
    if has_bingo:
        break
if has_bingo:
    remaining = board.remaining_sum()
    score = remaining*num
    print("Bingo on num:",ix)
    print("Board number:",board_ix)
    print("Score:",score)

Bingo!
Bingo on num: 29
Board number: 93
Score: 27027


### Part 2:
- Which board will win last?
- What is its score?

In [8]:
# Remake all the boards:
all_boards = []
for ix,board in enumerate(boards):
    all_boards.append(BingoBoard(board, board_num=ix))

In [9]:
# Loop through the input numbers and check them all
bingo_after = n_boards*[-1]
scores = n_boards*[0]
n_bingos = 0

for ix,num in enumerate(bingo_numbers):
    for board_ix,board in enumerate(all_boards):
        # If it already got bingo, skip it
        if bingo_after[board_ix] != -1:
            continue
        else:

            board.mark_number(num)
            has_bingo = board.check_bingo(False)

            # If it got bingo, record the score
            if has_bingo:
                remaining = board.remaining_sum()
                score = remaining*num
                bingo_after[board_ix] = ix
                scores[board_ix] = score
                n_bingos += 1
    if n_bingos == 100:
        final_num = num
        final_loop = ix
        print("Final bingo after num =",final_num)
        break
for ix,a in enumerate(bingo_after):
    if a == final_loop:
        board_ix = ix
print("Final board:",board_ix)
print("Score:",scores[board_ix])

Final bingo after num = 87
Final board: 21
Score: 36975
